In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_datasets as tfds

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


In [6]:
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(128)
ds_test = ds_train.prefetch(AUTOTUNE)

In [7]:
model = keras.Sequential(
    [
        keras.Input((28, 28, 1)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Flatten(),
        layers.Dense(10, activation="softmax"),
    ]
)

## Custom Training

In [8]:
num_epochs = 5
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
acc_metric = keras.metrics.SparseCategoricalAccuracy()


In [12]:
# Training Loop

for epoch in range(num_epochs):
  print(f'\n Start of Training Epoch {epoch}')

  for batch_idx, (x_batch, y_batch) in enumerate(ds_train):
    with tf.GradientTape() as tape:
      y_pred = model(x_batch, training=True)
      loss   = loss_fn(y_batch, y_pred)
    
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    acc_metric.update_state(y_batch, y_pred)

  train_acc = acc_metric.result()
  print(f"Accuracy over epoch {train_acc}")
  acc_metric.reset_states()



 Start of Training Epoch 0
Accuracy over epoch0.8990833163261414

 Start of Training Epoch 1
Accuracy over epoch0.9523333311080933

 Start of Training Epoch 2
Accuracy over epoch0.9695000052452087

 Start of Training Epoch 3
Accuracy over epoch0.9764500260353088

 Start of Training Epoch 4
Accuracy over epoch0.9807833433151245


In [13]:
# Test Loop
for batch_idx, (x_batch, y_batch) in enumerate(ds_test):
  y_pred = model(x_batch, training=True)
  acc_metric.update_state(y_batch, y_pred)

test-acc = acc_metric.result()
print(f"Accuracy over epoch {test_acc}")
acc_metric.reset_states()

Accuracy over epoch 0.984499990940094
